In [1]:
!git clone https://github.com/ThanhChinhBK/vietnews.git

Cloning into 'vietnews'...
remote: Enumerating objects: 143827, done.
remote: Total 143827 (delta 0), reused 0 (delta 0), pack-reused 143827 (from 1)
Receiving objects: 100% (143827/143827), 194.68 MiB | 25.45 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Updating files: 100% (150704/150704), done.


In [2]:
import os
import re

In [3]:
titles = []
abstracts = []
articles = []
source = '/kaggle/working/vietnews/data/train_tokenized'
count = 0

In [4]:
for filename in os.listdir(source):
    if (count == 90000):
        break
    file_path = os.path.join(source, filename)   # <-- full path
    if os.path.isfile(file_path):
        with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
            article = []
            for i, line in enumerate(f, start=1):
                text = re.sub(r'[()]', '', line.strip().lower())      # remove all ( )
                text = re.sub(r'[.?!:;,…]+$', '', text)
                if i == 1: 
                    titles.append(text)
                elif (i == 3):
                    abstracts.append(text)
                article.append(text)
            articles.append(' '.join(article))
    count+=1

In [5]:
from gensim.utils import simple_preprocess
import re

def split_sentences(text):
    # Very simple splitter: split on ., ! or ? and drop empties
    sents = re.sub(r"[\[\]\(\)\'\",.!?:]", " ", text)
    return sents.split()

corpus = []

# Articles contain many sentences, so break them up first
for art in articles:
    corpus.append(split_sentences(art))

In [6]:
from gensim.models import Word2Vec
# load = Word2Vec.load('word2vec_vi.model')
# if not load:
#     model = Word2Vec(
#         sentences=corpus,
#         vector_size=120,   # size of word embeddings
#         window=5,          # context window
#         min_count=2,       # ignore words that appear only once
#         workers=4,         # CPU cores
#         sg=1               # 1 = skip-gram; 0 = CBOW
#     )

#     model.save("word2vec_vi.model")
# else:
#     model = load
# model = Word2Vec(
#         sentences=corpus,
#         vector_size=120,   # size of word embeddings
#         window=5,          # context window
#         min_count=2,       # ignore words that appear only once
#         workers=4,         # CPU cores
#         sg=1               # 1 = skip-gram; 0 = CBOW
#     )
# model.save("word2vec_vi.model")
model = Word2Vec(
        sentences=corpus,
        vector_size=120,   # size of word embeddings
        window=5,          # context window
        min_count=2,       # ignore words that appear only once
        workers=4,         # CPU cores
        sg=1               # 1 = skip-gram; 0 = CBOW
    )
model.save("word2vec_vi.model")

In [7]:
print(model.wv.most_similar('công_an', topn=10))

[('csđt', 0.8039906620979309), ('pc45', 0.7666938900947571), ('pc02', 0.7593982219696045), ('pc04', 0.7527320981025696), ('pc46', 0.750519871711731), ('cshs', 0.7472440004348755), ('pc47', 0.746635913848877), ('catp', 0.7228366732597351), ('c47', 0.7191884517669678), ('caq', 0.7135355472564697)]


In [8]:
top10_freq = sorted(
    model.wv.key_to_index.keys(),
    key=lambda w: model.wv.get_vecattr(w, "count"),
    reverse=True
)[:10]

print(top10_freq)

['và', 'của', 'các', 'trong', 'được', 'có', 'là', 'đã', 'với', 'người']


In [9]:
word_vectors = model.wv

In [10]:
import numpy as np
stoi = {w: i+4 for i, w in enumerate(word_vectors.key_to_index)}
specials = ['<pad>', '<sos>', '<eos>', '<unk>']
for i,s in enumerate(specials): stoi[s] = i
itos = {i:s for s,i in stoi.items()}

embedding_dim = 120
embedding_matrix = np.zeros((len(stoi), embedding_dim))
for w, idx in stoi.items():
    if w in word_vectors:
        embedding_matrix[idx] = word_vectors[w]
    else:
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(embedding_dim,))

In [11]:
embedding_matrix

array([[ 0.34357445, -0.01881607,  0.44490534, ...,  1.49560657,
        -0.10458035, -0.33541327],
       [-0.45835551,  1.42854891,  0.28534884, ...,  0.53586882,
         0.22003493, -0.15998913],
       [ 0.63138211,  0.83444756, -0.33655864, ..., -1.15858037,
         0.37780703,  0.48300678],
       ...,
       [-0.01517487,  0.06603973, -0.10437654, ...,  0.01729302,
         0.04389772, -0.23397616],
       [-0.03448777,  0.11213791,  0.01215753, ...,  0.0509082 ,
         0.12976702, -0.17246613],
       [ 0.08124344,  0.10655062, -0.07755142, ...,  0.00344738,
         0.15523252, -0.24563707]])

In [12]:
from torch.utils.data import Dataset, DataLoader
class SummDataset(Dataset):
    def __init__(self, src_texts, target_texts, vocab, max_src=900, max_tgt=40):
        self.src = src_texts
        self.tgt = target_texts 
        self.vocab = vocab
        self.max_src = max_src # Max word in an article
        self.max_tgt = max_tgt # Max target words

    def encode(self, text, max_len):
        ids = [self.vocab.get(tok, self.vocab['<unk>']) for tok in text.split()]
        ids = ids[:max_len]
        return ids + [self.vocab['<pad>']]*(max_len - len(ids))

    def __getitem__(self, i):
        src_ids = self.encode(self.src[i], self.max_src)
        tgt_ids = [self.vocab['<sos>']] + \
                  self.encode(self.tgt[i], self.max_tgt-2) + \
                  [self.vocab['<eos>']]
        tgt_ids = tgt_ids + [self.vocab['<pad>']]*(self.max_tgt - len(tgt_ids))
        return torch.LongTensor(src_ids), torch.LongTensor(tgt_ids)

    def __len__(self): return len(self.src)

train_ds = SummDataset(articles, titles, stoi)
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)

In [13]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
device

device(type='cuda')

In [15]:
import torch.nn as nn

In [16]:
class ModelEncoder(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, embedding_weights):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_size, padding_idx=stoi['<pad>'])
        self.embedding.weight.data.copy_(torch.tensor(embedding_weights))
        self.embedding.weight.requires_grad = False
        
        # LSTM
        self.rnn = nn.LSTM(input_size= embedding_size, hidden_size=hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, hidden_size) # A_foward + A_backward as input and then compress to one
        
    def forward(self, src):
        emb = self.embedding(src)
        outputs, (h_n, c_n) = self.rnn(emb)   # outputs: [B,S,2H], h_n/c_n: [2,B,H]

        # Ghép 2 hướng lại thành [B, 2H]
        h_cat = torch.cat((h_n[-2], h_n[-1]), dim=1)   # [B, 2H]
        c_cat = torch.cat((c_n[-2], c_n[-1]), dim=1)   # [B, 2H]

        # Chiếu xuống H và thêm chiều num_layers=1
        h_final = torch.tanh(self.fc(h_cat)).unsqueeze(0)  # [1,B,H]
        c_final = torch.tanh(self.fc(c_cat)).unsqueeze(0)  # [1,B,H]

        return self.fc(outputs), (h_final, c_final)

In [17]:
class ModelAttention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.attn = nn.Linear(hidden_size*2, hidden_size) # Concat Encoder + Decoder
        self.v = nn.Linear(hidden_size, 1, bias=False)
        self.softmax = nn.Softmax()

    def forward(self, decoder_hidden, encoder_outputs):
        a = encoder_outputs                             # [B, S, H]
        h, _ = decoder_hidden               # take only the hidden state
        # h shape: [num_layers, B, H]  (here num_layers=1 for decoder)
        h = h.permute(1, 0, 2)              # -> [B, 1, H]
        B, S, H = encoder_outputs.size()
        s = h.repeat(1, S, 1)               # [B, S, H]

        energy = torch.relu(self.attn(torch.cat((s, a), dim=2)))  # [B, S, H]
        scores = self.v(energy).squeeze(2)              # [B, S]
        attn_weights = torch.softmax(scores, dim=1)     # [B, S]
        context = torch.bmm(attn_weights.unsqueeze(1), a)  # [B, 1, H]
        return context, attn_weights


In [18]:
class ModelDecoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=stoi['<pad>'])
        self.rnn = nn.LSTM(hidden_size + emb_size, hidden_size, batch_first=True)
        self.fc_out = nn.Linear(hidden_size*2 + emb_size, vocab_size)
        self.attention = ModelAttention(hidden_size)

    def forward(self, input_token, hidden, encoder_outputs):
        # input_token: [B] current word index
        emb = self.embedding(input_token).unsqueeze(1)     # [B,1,E]
        context, attn_weights = self.attention(hidden, encoder_outputs)
        rnn_input = torch.cat((emb, context), dim=2)       # [B,1,E+H]
        output, hidden = self.rnn(rnn_input, hidden)       # output: [B,1,H]
        concat = torch.cat((output, context, emb), dim=2)  # [B,1,H+H+E]
        prediction = self.fc_out(concat).squeeze(1)        # [B,vocab]
        return prediction, hidden, attn_weights


In [19]:
class SummarizationModel(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt, teacher_forcing_ratio=0.5):
        encoder_outputs, hidden = self.encoder(src)
        input_token = tgt[:,0]        # usually <sos>
        outputs = []
        for t in range(1, tgt.size(1)):
            output, hidden, _ = self.decoder(input_token, hidden, encoder_outputs)
            outputs.append(output.unsqueeze(1))
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            input_token = tgt[:,t] if teacher_force else output.argmax(1)
        return torch.cat(outputs, dim=1)   # [B, T-1, vocab]


In [20]:
encoder = ModelEncoder(vocab_size=len(stoi), embedding_size=120, hidden_size=256, embedding_weights=embedding_matrix)
decoder = ModelDecoder(len(stoi), emb_size=120, hidden_size=256)
sum_model   = SummarizationModel(encoder, decoder).to(device)

In [21]:
criterion = nn.CrossEntropyLoss(ignore_index=stoi['<pad>'])
optimizer = torch.optim.Adam(sum_model.parameters(), lr=3e-4)

In [22]:
def train_one_epoch(model, dataloader, optimizer, criterion, device, teacher_forcing_ratio=0.5):
    model.train()
    total_loss = 0

    for src, tgt in dataloader:           # src: [B,S], tgt: [B,T]
        src, tgt = src.to(device), tgt.to(device)

        optimizer.zero_grad()

        # model returns predictions for each target step except the first <sos>
        output = model(src, tgt, teacher_forcing_ratio)
        # output shape: [B, T-1, vocab]

        # Align target: skip first token (<sos>)
        target = tgt[:, 1:]                # [B, T-1]

        # Flatten for CrossEntropy: [(B*(T-1)), vocab]
        loss = criterion(
            output.reshape(-1, output.size(-1)),
            target.reshape(-1)
        )

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # optional: gradient clipping
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [23]:
NUM_EPOCHS = 15

for epoch in range(1, NUM_EPOCHS + 1):
    train_loss = train_one_epoch(sum_model, train_loader,
                                 optimizer, criterion, device)
    # val_loss = evaluate(model, val_loader, criterion, device)  # if you have val set
    if (train_loss <= 0.2):
        break
    print(f"Epoch {epoch:02d}: train loss = {train_loss:.4f}")


Epoch 01: train loss = 2.8282
Epoch 02: train loss = 0.4800
Epoch 03: train loss = 0.2210


In [24]:
torch.save(sum_model.state_dict(), "summarization_model_weights.pth")

In [25]:
def encode_text(text, vocab, max_len=800):
    ids = [vocab.get(tok, vocab['<unk>']) for tok in text.split()]
    ids = ids[:max_len]
    return torch.LongTensor([ids + [vocab['<pad>']] * (max_len - len(ids))])


In [26]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for src, tgt in dataloader:
            src, tgt = src.to(device), tgt.to(device)
            output = model(src, tgt)  # no teacher forcing
            target = tgt[:, 1:]
            loss = criterion(output.reshape(-1, output.size(-1)),
                             target.reshape(-1))
            total_loss += loss.item()
    return total_loss / len(dataloader)


In [27]:
def generate_title_lstm(model, article_text, max_len=40):
    model.eval()
    with torch.no_grad():
        # encode_text already returns [1, max_len]
        src = encode_text(article_text, stoi).to(device)

        encoder_outputs, hidden = model.encoder(src)

        input_token = torch.tensor([stoi['<sos>']], device=device)
        generated = []

        for _ in range(max_len):
            output, hidden, _ = model.decoder(input_token, hidden, encoder_outputs)
            next_token = output.argmax(1)
            if next_token.item() == stoi['<eos>']:
                break
            generated.append(next_token.item())
            input_token = next_token

    words = [itos[i] for i in generated]
    return " ".join(words), words


In [28]:
new_data = '''Bản_án cho đối_tượng giả_danh công_an để lừa_đảo

Ngày 25/2 , TAND TP. Đà_Nẵng tuyên_phạt Hồ_Xuân_Huy ( SN 1994 ) , ngụ quận Hải_Châu , 12 năm tù về tội Lừa_đảo chiếm_đoạt tài_sản .

Theo lời khai của Huy tại phiên_toà , để có tiền sử_dụng cá_nhân , Huy “ nổ ” là sĩ_quan cục Phòng_chống ma_tuý của bộ Công_an đóng tại TP. Đà_Nẵng , có nguồn mua ô_tô thanh_lý giá rẻ , và khả_năng chạy việc vào ngành công_an .
Chỉ với lời “ nổ ” này , từ tháng 10/2016 đến 9/2017 , nhiều người đã bị lừa_đảo với tổng_số tiền 3,2 tỷ đồng .
Trong đó , người bị Huy lừa nhiều nhất là vợ_chồng ông Bảo_Th . , ngụ quận Hải_Châu .
Huy giới_thiệu với cặp vợ_chồng này mình có suất mua ô_tô thanh_lý giá rẻ và rủ họ mua cùng .
Tin lời , vợ_chồng ông Th .
đưa cho Huy hơn 1 tỷ đồng .
Cùng thủ_đoạn , Huy lừa thêm ông Nguyễn_Tấn_T. 970 triệu đồng , Lê_Quốc_Th .
400 triệu đồng , Trần_Nhật_S. 300 triệu đồng …
Sau chiêu_thức mua xe thanh_lý , Huy chuyển sang giả_vờ có khả_năng xin việc vào ngành công_an .
Với chiêu_thức này , Huy lừa vợ_chồng ông Đinh_Ngọc_H. 250 triệu đồng .
Ngoài_ra , Huy hứa_hẹn , tháng 3/2017 sẽ đưa kết_quả cho con ông H. đi làm_việc .
Tuy_nhiên , sau nhiều lần hẹn mà không có quyết_định tuyển_dụng , ông H. đã gửi đơn tố_cáo đến cơ_quan Công_an .
Từ đó , những hành_vi sai_trái của Huy lần_lượt được truy ra .

Huy tại phiên_toà .
'''

In [29]:
sample_article = articles[0]  # or any new article string
title_pred, words = generate_title_lstm(sum_model, new_data)
print("Predicted title:", title_pred)

Predicted title: <unk> cho đối_tượng giả_danh công_an để lừa_đảo <unk> 25/2 <unk> <unk> <unk> <unk> tuyên_phạt <unk> <unk> <unk> 1993 <unk> <unk> ngụ quận <unk> <unk> 12 năm tù <unk> tù <unk> tù <unk>
